In [1]:
import torch
import torch.nn as nn
from torch.nn import functional as F
from torch.nn.functional import dropout
import re

In [2]:
# Гиперпараметры
batch_size = 64 # сколько независимых последовательностей мы будем обрабатывать параллельно
block_size = 256 # Какова максимальная длина контекста для прогнозов
max_iters = 5000
eval_interval = 500
learning_rate = 3e-4
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 200
n_embd = 384
n_head = 6
n_layer = 6
dropout = 0.2

In [3]:
with open (r'D:\\Python\Projects\HW2last\pythonProject1\biesy.txt', encoding='utf-8') as f:
    text = f.read()
    text = re.sub(r'[^А-я—.,?;:! ]', '', text)

In [4]:
chars = sorted(list(set(text))) # список всех символов из нашего файла
vocab_size = len(chars)
print(''.join(chars))
print(vocab_size)

 !,.:;?АБВГДЕЖЗИКЛМНОПРСТУФХЦЧШЭЮЯабвгдежзийклмнопрстуфхцчшщъыьэюя—
67


In [5]:
st = { ch:i for i,ch in enumerate(chars) }
it = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [st[c] for c in s] # из строки выводим список целых чисел
decode = lambda l: ''.join([it[i] for i in l]) # из списка целых чисел получаем обратно строку

In [6]:
# для примера выведем слово привет
print(encode("привет"))
print(decode(encode("привет")))

[49, 50, 42, 36, 39, 52]
привет


In [7]:
# создаем тензор из наших данных
data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape, data.dtype)
print(data[:100])

torch.Size([1297789]) torch.int64
tensor([10, 45, 34, 36, 34,  0, 49, 39, 50, 36, 34, 65,  9, 46, 39, 51, 52, 48,
         0, 36, 36, 39, 38, 39, 47, 42, 65,  4,  0, 47, 39, 51, 44, 48, 45, 62,
        44, 48,  0, 49, 48, 38, 50, 48, 35, 47, 48, 51, 52, 39, 43,  0, 42, 41,
         0, 35, 42, 48, 37, 50, 34, 54, 42, 42,  0, 46, 47, 48, 37, 48, 57, 52,
        42, 46, 48, 37, 48,  0, 23, 52, 39, 49, 34, 47, 34,  0, 24, 50, 48, 54,
        42, 46, 48, 36, 42, 57, 34,  0,  9, 39])


In [8]:
# разделяем на тренировочную и проверочную выборки
n = int(0.9*len(data))
train_data = data[:n]
val_data = data[n:]

In [9]:
# загрузка данных
def get_batch(split):

    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

In [10]:
# функция потерь
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

In [11]:
class Head(nn.Module):
    """ одна голова самовнимания """
    
    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))
        self.dropout = nn.Dropout(dropout)
    def forward(self, x):
        B,T,C = x.shape
        k = self.key(x) # (B, T, C)
        q = self.query(x) # (B, T, C)
        # вычисляем баллы внимания
        wei = q @ k.transpose(-2,-1) * C**-0.5 # (B, T, C) @ (B, C, T) --> (B, T, T)
        wei = wei.masked_fill(self.tril[:T,:T] == 0, float('-inf')) # (B, T, T)
        wei = F.softmax(wei, dim=-1) # (B, T, T)
        wei = self.dropout(wei)
        # выполняем взвешенное агрегирование значений
        v = self.value(x) # (B,T,C)
        out = wei @ v # (B, T, T) @ (B, T, C) --> (B, T, C)
        return out

In [12]:
class MultiHeadAttention(nn.Module):
    """ несколько голов внутреннего внимания параллельно"""

    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(n_embd, n_embd)
        self.dropout = nn.Dropout(dropout)
    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.proj(out)
        return out

In [13]:
class FeedForward(nn.Module):
    """ простой линейный слой, за которым следует нелинейность """

    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout),
        )

    def forward(self, x):
        return self.net(x)

In [14]:
class Block(nn.Module):
    """ Блок трансформера: связь с последующим вычислением """

    def __init__(self, n_embd, n_head):
        # n_embd - размерность встраивания, n_head - количество голов, которые нам нужны
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedForward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)
    def forward(self, x):
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x

In [15]:
class BigramLanguageModel(nn.Module):

    def __init__(self):
        super().__init__()
        #
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd) # последний слой нормализации
        self.lm_head = nn.Linear(n_embd, vocab_size)

    def forward(self, idx, target=None):
        B, T = idx.shape
        #idx и targets оба (B, T) целочисленные тензоры
        tok_emb = self.token_embedding_table(idx) # (B,T,C)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device)) # (T, C)
        x = tok_emb + pos_emb # (B,T,C)
        x = self.blocks(x) # (B,T,C)
        logits = self.lm_head(x) # (B,T,vocab_size)

        if target is None:
            loss=None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            target = target.view(B*T)
            loss = F.cross_entropy(logits, target)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx — это (B, T) массив индексов в текущем контексте
        for _ in range(max_new_tokens):
            # обрезаем idx до последних токенов block_size
            idx_cond = idx[:, -block_size:]
            # получаем прогнозы
            logits, loss = self(idx_cond)
            # сосредоточимся только на последнем временном шаге
            logits = logits[:,-1,:] #  (B, C)
            # применим softmax для получения вероятностей
            probs = F.softmax(logits, dim=1) # (B, C)
            # выборка для раздачи
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # добавляем выборочный индекс к работающей последовательности
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

In [ ]:
model = BigramLanguageModel()
m = model.to(device)

optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):

    if iter % eval_interval == 0:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    # выборка пакета данных
    xb, yb = get_batch('train')

    # оценка потерь
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

step 0: train loss 4.5751, val loss 4.5657


In [17]:
context = torch.zeros((1, 1), dtype=torch.long, device=device)
print(decode(m.generate(context, max_new_tokens=300)[0].tolist()))

 открытыми дама руками, не давеча, напримешниколевы, стадал мне, из комнатыми, что тот он не узнал тебя опять необычайностей, то, про слеза. Да и могу же я   распришил его, Варваре Петровне мне покраснела, которая укрепись: сначала была поглядели ей, несмотря на читанность. Я не любила одно лишь это,
